In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os.path as path
import os
#up = path.abspath(path.join('FastRCNNPredictor.ipynb' ,"../../../.."))

Mounted at /content/drive


In [ ]:
working_dir = '/content/drive/Shareddrives/NN_Project/NN_Final/frcnn/'
data_file = '/content/drive/Shareddrives/NN_Project/NN_Final/frcnn/data_file_final_annot.csv'
data_folder = '/content/drive/Shareddrives/NN_Project/NN_Final/SortedClasses/combined_images/'
model_path = '/content/drive/Shareddrives/NN_Project/NN_Final/SortedClasses/model/models'
num_classes = 6

In [ ]:
os.chdir(working_dir)

In [ ]:
import numpy as np
import os
import torch
import pycocotools
import torch.utils.data
from PIL import Image
import pandas as pd
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
from  PIL import ImageDraw
import os.path as path
import re

In [ ]:
def parse_one_annot(path_to_data_file, filename):
    data = pd.read_csv(path_to_data_file)
    print(filename)
    boxes_array = data[data["filename"] == filename][["xmin", "ymin", "xmax", "ymax"]].values
    print(boxes_array)
    return boxes_array

In [ ]:
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    print("inside list of files")
    print(dirName)
    print(listOfFile)
    
    print("Full paths")
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        # print("printing full path ", fullPath)
        if os.path.isdir(fullPath):
          allFiles = allFiles + getListOfFiles(fullPath)
        else:
          # print("inside else part")
          fullPath = fullPath.replace(data_folder, '')
          # print(fullPath)
          allFiles.append(fullPath)  
    return allFiles    

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, root, data_file, transforms=None):
        self.root = root
        self.transforms = transforms
        files = getListOfFiles(os.path.join(root))
        print('printing files')
        print(files)
        # p = re.compile('batch')
        # l2 = [ s for s in files if p.match(s) ]
        print("printing l2")
        # print(l2)
        self.imgs = sorted(files)
        self.path_to_data_file = data_file
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.imgs[idx])
        self.imgs[idx] = self.imgs[idx]
        img = Image.open(img_path).convert("RGB")
        box_list = parse_one_annot(self.path_to_data_file, self.imgs[idx])
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
                 img, target = self.transforms(img, target)
        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [ ]:
dataset = Dataset(root= data_folder, data_file= data_file)
dataset

inside list of files
/content/drive/Shareddrives/NN_Project/NN_Final/SortedClasses/combined_images/
['paper327.jpg', 'paper579.jpg', 'paper580.jpg', 'cardboard131.jpg', 'cardboard132.jpg', 'cardboard133.jpg', 'cardboard134.jpg', 'cardboard137.jpg', 'cardboard136.jpg', 'cardboard140.jpg', 'cardboard142.jpg', 'cardboard144.jpg', 'cardboard143.jpg', 'cardboard145.jpg', 'cardboard146.jpg', 'cardboard148.jpg', 'cardboard147.jpg', 'cardboard149.jpg', 'cardboard150.jpg', 'cardboard151.jpg', 'cardboard152.jpg', 'cardboard153.jpg', 'cardboard154.jpg', 'cardboard156.jpg', 'cardboard157.jpg', 'cardboard160.jpg', 'cardboard161.jpg', 'cardboard162.jpg', 'cardboard163.jpg', 'cardboard164.jpg', 'cardboard166.jpg', 'cardboard167.jpg', 'cardboard169.jpg', 'cardboard168.jpg', 'cardboard171.jpg', 'cardboard170.jpg', 'cardboard173.jpg', 'cardboard172.jpg', 'cardboard178.jpg', 'cardboard179.jpg', 'cardboard180.jpg', 'cardboard182.jpg', 'cardboard181.jpg', 'cardboard184.jpg', 'cardboard185.jpg', 'cardboard1

In [ ]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
    return model

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
dataset = Dataset(root = data_folder, 
                  data_file = data_file, 
                  transforms = get_transform(train=True))
dataset_test = Dataset(root = data_folder, data_file = data_file, transforms = get_transform(train=False))

inside list of files
/content/drive/Shareddrives/NN_Project/NN_Final/SortedClasses/combined_images/
['paper327.jpg', 'paper579.jpg', 'paper580.jpg', 'cardboard131.jpg', 'cardboard132.jpg', 'cardboard133.jpg', 'cardboard134.jpg', 'cardboard137.jpg', 'cardboard136.jpg', 'cardboard140.jpg', 'cardboard142.jpg', 'cardboard144.jpg', 'cardboard143.jpg', 'cardboard145.jpg', 'cardboard146.jpg', 'cardboard148.jpg', 'cardboard147.jpg', 'cardboard149.jpg', 'cardboard150.jpg', 'cardboard151.jpg', 'cardboard152.jpg', 'cardboard153.jpg', 'cardboard154.jpg', 'cardboard156.jpg', 'cardboard157.jpg', 'cardboard160.jpg', 'cardboard161.jpg', 'cardboard162.jpg', 'cardboard163.jpg', 'cardboard164.jpg', 'cardboard166.jpg', 'cardboard167.jpg', 'cardboard169.jpg', 'cardboard168.jpg', 'cardboard171.jpg', 'cardboard170.jpg', 'cardboard173.jpg', 'cardboard172.jpg', 'cardboard178.jpg', 'cardboard179.jpg', 'cardboard180.jpg', 'cardboard182.jpg', 'cardboard181.jpg', 'cardboard184.jpg', 'cardboard185.jpg', 'cardboard1

In [ ]:
print(dataset.imgs)

['-vector-id134024774.jpg', '0c3c26e120fc8ea4e89f78969e91b7dd.jpg', '1-consumerelec.jpg', '1600968615238.jpg', '160267825-scrap-yard-printed-circuit-board-electronic-waste-for-recycling-with-selective-focus-main-board-elec.jpg', '5-1-1-1.jpg', '7-ways-to-use-less-paper_paper-globe-discarded-as-trash.jpg', 'E-waste.jpg', 'Eco_Music_800x450.jpg', 'Example-of-e-waste-in-Category-3-ICT-a-used-mobile-phones-and-b-used-monitors_Q320.jpg', 'Galsswaste.jpg', 'Glass(1).jpg', 'Glass-Bottles.jpg', 'Recycling-1.jpg', 'Recycling-Paper.jpg', '_105130816_vapi_2-1.png', 'abandoned-and-rusted-laptop-lying-on-riverbed-picture-id108162816.jpg', 'abandoned-computers-picture-id168239718.jpg', 'abandoned-electronic-equipments-in-recycling-plant-picture-id958247858.jpg', 'abandoned-pc-monitor-closeup-picture-id157696060.jpg', 'abandoned-pc-screen-picture-id157696061.jpg', 'abnormal-deformed-ripe-strawberries-latest-trend-eating-ugly-fruits-vegetables-reduction-food-organic-waste-211927045.jpg', 'an-ewaste-bu

In [ ]:
print(len(dataset))

856


In [ ]:
print(len(dataset_test))

856


In [ ]:
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
testing_num = int(len(dataset)*0.2)
print("Testing num ", testing_num)
dataset = torch.utils.data.Subset(dataset, indices[:-testing_num])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-testing_num:])
data_loader = torch.utils.data.DataLoader(
              dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(
         dataset_test, batch_size=1, shuffle=False, num_workers=4,
         collate_fn=utils.collate_fn)
print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

Testing num  171
We have: 856 examples, 685 are training and 171 testing


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
print("Data loader train ", len(data_loader))
print("data loader test ", len(data_loader_test))

Data loader train  343
data loader test  171


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model(num_classes)
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
num_epochs = 7
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


ugly-food-deformed-organic-carrots-blue-background-deformed-products-food-waste-problem-concept-ugly-food-deformed-organic-198203484.jpg
detail-colorful-conductors-discarded-computer-electronic-parts-environmental-contamination-danger-selective-focus-metal-160437405.jpg
book-5081216__480.jpg
[[373.40425532 130.85106383 895.21276596 737.55319149]]
[[ 363.82978723  155.32978723 1062.76595745  787.25531915]
 [ 395.74468085  224.4787234   962.76595745  753.21276596]
 [ 311.70212766  237.24468085  876.59574468  752.14893617]
 [ 359.57446809  350.0106383   836.17021277  637.25531915]
 [ 136.17021277  148.94680851 1003.19148936  693.63829787]
 [ 304.25531915   94.69148936  879.78723404  705.34042553]
 [  77.65957447  118.09574468 1027.65957447  861.72340426]
 [ 235.10638298   73.41489362  854.25531915  705.34042553]
 [ 388.29787234   92.56382979  853.19148936  733.        ]]
[[  72.34042553    0.         1231.91489362  715.95744681]]
damaged-compact-disc-secure-backup-disposal-digital-persona

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ugly-food-vegetable-deformed-organic-ripe-tomato-white-background-buying-misshapen-produce-as-waste-problem-slightly-lower-171356895.jpg
[[ 241.4893617   192.56382979 1084.04255319  792.57446809]]
an-ewaste-bucket-during-the-warsaw-recycling-days-on-september-18-in-picture-id608729994.jpg
[[1.04170213e+02 1.36859043e+02 7.15356383e+02 5.65882979e+02]
 [1.99388298e+02 3.67579787e+01 8.96840426e+02 6.83888298e+02]
 [4.12611702e+02 1.25465426e+02 7.21053191e+02 6.87957447e+02]
 [1.50558511e+02 2.90672872e+02 7.29191489e+02 5.12984043e+02]
 [2.35196809e+02 2.45098404e+02 6.86872340e+02 5.62627660e+02]
 [2.77515957e+02 2.64630319e+02 7.44654255e+02 4.99962766e+02]
 [1.96946809e+02 1.07154255e+01 6.45367021e+02 5.29260638e+02]
 [3.58898936e+02 1.35638298e-01 7.25936170e+02 4.93452128e+02]
 [2.46590426e+02 0.00000000e+00 7.91856383e+02 4.98470745e+02]]
Epoch: [0]  [  0/343]  eta: 5:09:53  lr: 0.000020  loss: 2.4992 (2.4992)  loss_classifier: 1.8883 (1.8883)  loss_box_reg: 0.0028 (0.0028)  los

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


syringe waste_49.png
[[ 154.15159574  241.26861702  939.39361702  973.96010638]
 [ 313.47606383  406.80053191  958.01595745  800.15159574]
 [  63.10904255   79.875      1448.40425532 1123.97340426]]
tv-computer-screens-waste-disposal-and-recycling-picture-id92218664.jpg
[[  0.         154.76329787 831.73404255 657.84574468]
 [218.92021277 141.74202128 850.45212766 650.5212766 ]
 [128.58510638  13.15691489 750.35106383 531.70212766]
 [464.69680851 223.93882979 759.30319149 592.06117021]
 [459.         149.06648936 747.09574468 513.79787234]
 [271.00531915  56.28989362 747.90957447 519.49468085]
 [421.56382979  44.8962766  738.95744681 539.84042553]
 [  1.62765957  97.79521277 773.13829787 508.91489362]]
Epoch: [0]  [ 60/343]  eta: 3:32:12  lr: 0.000896  loss: 0.2849 (0.9632)  loss_classifier: 0.0310 (0.3944)  loss_box_reg: 0.0067 (0.0137)  loss_objectness: 0.0252 (0.1887)  loss_rpn_box_reg: 0.2201 (0.3665)  time: 43.9176  data: 0.0061  max mem: 0
cardboard164.jpg
[[ 54.46808511   2.3829

In [ ]:
torch.save(model.state_dict(), model_path)

In [ ]:
loaded_model = get_model(num_classes = num_classes)
loaded_model.load_state_dict(torch.load(model_path))

In [ ]:
idx = 1
img, _ = dataset_test[idx]
label_boxes = np.array(dataset_test[idx][1]["boxes"])
loaded_model.eval()
with torch.no_grad():
    prediction = loaded_model([img])
image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
draw = ImageDraw.Draw(image)
for elem in range(len(label_boxes)):
    draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
    (label_boxes[elem][2], label_boxes[elem][3])], 
    outline ="green", width =3)
for element in range(len(prediction[0]["boxes"])):
    boxes = prediction[0]["boxes"][element].cpu().numpy()
    score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                    decimals= 4)
    if score > 0.8:
        draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
        outline ="red", width =3)
        draw.text((boxes[0], boxes[1]), text = str(score))
image